# March Madness Prediction

## Overview

### Goal
Submissions are based on the Brier Score, the goal will be to minimize the brier score between the predicted probabilities and the actual game outcomes. The Brier score measures the accuracy of probablistic predition, in this case the mean square error. 

The brier score can be thought of as a cost function that measures the average squared difference between the predicted probabilities and the actual outcomes.

$$
Brier = \frac{1}{N} \sum_{i=1}^{N} (p_i - o_i)^2
$$

where $p_i$ is the predicted probability of the event and $o_i$ is the actual outcome. The Brier score can span across all items in a set of N predictions.

Therefore, minimizing the Brier score will result in a more accurate prediction.


## Import Libraries
Numpy for numerical operations
Pandas for data manipulation
Matplotlib, Seaborn, Plotly for plotting


In [1]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as sp
import xgboost as xgb
import sklearn as sk

## Load Data

Set up a data dictionary that will store the data for each file, this will make it easier to access data from the csvs. Not all files are used in the prediction process, but they are included for completeness.


In [3]:
data_dir = 'data/'
data = {}

files = glob.glob(data_dir + '*.csv')
for file in files:
    filename = file.split('/')[-1].split('\\')[-1].split('.')[0]
    try:
        data[filename] = pd.read_csv(file, encoding='latin-1')
    except Exception as e:
        print(f"Warning: Could not load {filename}: {e}")



Now that we have the data loaded, lets start by loading team and team spelling data. Here we will be using the team and team spelling data from the regular season and tournament. Combining women and mens data to get a complete dataset.


In [8]:
teams = pd.concat([data['MTeams'], data['WTeams']])
teams_spelling = pd.concat([data['MTeamSpellings'], data['WTeamSpellings']])
teams_spelling = teams_spelling.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
teams_spelling.columns = ['TeamID', 'TeamNameCount']
teams = pd.merge(teams, teams_spelling, how='left', on=['TeamID'])
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season,TeamNameCount
0,1101,Abilene Chr,2014.0,2025.0,3
1,1102,Air Force,1985.0,2025.0,2
2,1103,Akron,1985.0,2025.0,1
3,1104,Alabama,1985.0,2025.0,1
4,1105,Alabama A&M,2000.0,2025.0,2



Lets add the season and turney reluts, both compact and detailed. There is a chance we only end up using the detailed results since there is more information available and we will be able to derive more features from the detailed results.

In [18]:
season_compact_results = pd.concat([data['MRegularSeasonCompactResults'], data['WRegularSeasonCompactResults']]).assign(ST='S')
season_detailed_results = pd.concat([data['MRegularSeasonDetailedResults'], data['WRegularSeasonDetailedResults']]).assign(ST='S')
tourney_compact_results = pd.concat([data['MNCAATourneyCompactResults'], data['WNCAATourneyCompactResults']]).assign(ST='T')
tourney_detailed_results = pd.concat([data['MNCAATourneyDetailedResults'], data['WNCAATourneyDetailedResults']]).assign(ST='T')

Now have the season and touurney results, both compact and detailed, Lets take a look at the data. We can do this by creating a summary of the datasets. This will help us understand the shape of the data and the columns that are available. Taking a look at the head of the datasets will give us a better understanding of the data as well.

In [19]:
# Create a summary of the datasets
summary = pd.DataFrame({
    'Dataset': [
        'Regular Season Compact',
        'Regular Season Detailed',
        'Tournament Compact',
        'Tournament Detailed'
    ],
    'Shape': [
        season_compact_results.shape,
        season_detailed_results.shape,
        tourney_compact_results.shape,
        tourney_detailed_results.shape
    ],
    'Columns': [
        len(season_compact_results.columns),
        len(season_detailed_results.columns),
        len(tourney_compact_results.columns),
        len(tourney_detailed_results.columns)
    ],
    'Sample Years': [
        f"{season_compact_results['Season'].min()}-{season_compact_results['Season'].max()}",
        f"{season_detailed_results['Season'].min()}-{season_detailed_results['Season'].max()}",
        f"{tourney_compact_results['Season'].min()}-{tourney_compact_results['Season'].max()}",
        f"{tourney_detailed_results['Season'].min()}-{tourney_detailed_results['Season'].max()}"
    ]
})

# Display the summary with nice formatting
display(summary.style.set_properties(**{
    'border-color': 'black',
    'border-style': 'solid',
    'border-width': '1px',
    'text-align': 'center'
}))

# If you want to see the first few rows of each dataset, you can also do:
# print("\nSample of Regular Season Compact Results:")
# display(season_compact_results.head(3))

# print("\nSample of Regular Season Detailed Results:")
# display(season_detailed_results.head(3))

# print("\nSample of Tournament Compact Results:")
# display(tourney_compact_results.head(3))

# print("\nSample of Tournament Detailed Results:")
# display(tourney_detailed_results.head(3))

,Dataset,Shape,Columns,Sample Years
0,Regular Season Compact,"(329125, 9)",9,1985-2025
1,Regular Season Detailed,"(199757, 35)",35,2003-2025
2,Tournament Compact,"(4168, 9)",9,1985-2024
3,Tournament Detailed,"(2276, 35)",35,2003-2024


In [20]:
# Load other data that might prove useful
slots = pd.concat([data['MNCAATourneySlots'], data['WNCAATourneySlots']])
seeds = pd.concat([data['MNCAATourneySeeds'], data['WNCAATourneySeeds']])
seeds['SeedValue'] = seeds['Seed'].str.extract(r'(\d+)').astype(int)
seeds_dict = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in seeds[['Season', 'Seed', 'TeamID']].values}
game_cities = pd.concat([data['MGameCities'], data['WGameCities']])
seasons = pd.concat([data['MSeasons'], data['WSeasons']])

# Combine all game results
all_compact_results = pd.concat([season_compact_results, tourney_compact_results])
all_detailed_results = pd.concat([season_detailed_results, tourney_detailed_results])


## Feature Engineering



Using the data from the compact and detailed results, we can derive more features that will be used to train the model. For example shooting percentages, score difference, and home advantage are some statistics that are often mentioned in the basketball community as being important.

In [21]:
# Add derived features to compact results
all_compact_results['ScoreDiff'] = all_compact_results['WScore'] - all_compact_results['LScore']
all_compact_results['HomeAdvantage'] = all_compact_results['WLoc'].map({'H': 1, 'N': 0, 'A': -1})

# Add derived features to detaifled results
all_detailed_results['ScoreDiff'] = all_detailed_results['WScore'] - all_detailed_results['LScore']
all_detailed_results['HomeAdvantage'] = all_detailed_results['WLoc'].map({'H': 1, 'N': 0, 'A': -1})

    # Calculate shooting percentages (handling division by zero)
all_detailed_results['WFGPct'] = np.where(all_detailed_results['WFGA'] > 0, 
                                        all_detailed_results['WFGM'] / all_detailed_results['WFGA'], 0)
all_detailed_results['WFG3Pct'] = np.where(all_detailed_results['WFGA3'] > 0, 
                                        all_detailed_results['WFGM3'] / all_detailed_results['WFGA3'], 0)
all_detailed_results['WFTPct'] = np.where(all_detailed_results['WFTA'] > 0, 
                                        all_detailed_results['WFTM'] / all_detailed_results['WFTA'], 0)
all_detailed_results['LFGPct'] = np.where(all_detailed_results['LFGA'] > 0, 
                                        all_detailed_results['LFGM'] / all_detailed_results['LFGA'], 0)
all_detailed_results['LFG3Pct'] = np.where(all_detailed_results['LFGA3'] > 0, 
                                        all_detailed_results['LFGM3'] / all_detailed_results['LFGA3'], 0)
all_detailed_results['LFTPct'] = np.where(all_detailed_results['LFTA'] > 0, 
                                        all_detailed_results['LFTM'] / all_detailed_results['LFTA'], 0)

# Add statistical differences
all_detailed_results['ReboundDiff'] = (all_detailed_results['WOR'] + all_detailed_results['WDR']) - \
                                    (all_detailed_results['LOR'] + all_detailed_results['LDR'])
all_detailed_results['AssistDiff'] = all_detailed_results['WAst'] - all_detailed_results['LAst']
all_detailed_results['TurnoverDiff'] = all_detailed_results['WTO'] - all_detailed_results['LTO']
all_detailed_results['StealDiff'] = all_detailed_results['WStl'] - all_detailed_results['LStl']
all_detailed_results['BlockDiff'] = all_detailed_results['WBlk'] - all_detailed_results['LBlk']
all_detailed_results['FoulDiff'] = all_detailed_results['WPF'] - all_detailed_results['LPF']

# Add seed information to tournament games
tourney_compact = all_compact_results[all_compact_results['ST'] == 'T'].copy()
tourney_detailed = all_detailed_results[all_detailed_results['ST'] == 'T'].copy()

# Add winner seeds
tourney_compact = pd.merge(
    tourney_compact,
    seeds[['Season', 'TeamID', 'SeedValue']],
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
)
tourney_compact.rename(columns={'SeedValue': 'WSeedValue'}, inplace=True)
tourney_compact.drop('TeamID', axis=1, inplace=True)

tourney_detailed = pd.merge(
    tourney_detailed,
    seeds[['Season', 'TeamID', 'SeedValue']],
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
)
tourney_detailed.rename(columns={'SeedValue': 'WSeedValue'}, inplace=True)
tourney_detailed.drop('TeamID', axis=1, inplace=True)

# Add loser seeds
tourney_compact = pd.merge(
    tourney_compact,
    seeds[['Season', 'TeamID', 'SeedValue']],
    how='left',
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
)
tourney_compact.rename(columns={'SeedValue': 'LSeedValue'}, inplace=True)
tourney_compact.drop('TeamID', axis=1, inplace=True)

tourney_detailed = pd.merge(
    tourney_detailed,
    seeds[['Season', 'TeamID', 'SeedValue']],
    how='left',
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
)
tourney_detailed.rename(columns={'SeedValue': 'LSeedValue'}, inplace=True)
tourney_detailed.drop('TeamID', axis=1, inplace=True)

    # Calculate seed difference (lower is better in seeding, so LSeed - WSeed is positive if favorite won)
tourney_compact['SeedDiff'] = tourney_compact['LSeedValue'] - tourney_compact['WSeedValue']
tourney_detailed['SeedDiff'] = tourney_detailed['LSeedValue'] - tourney_detailed['WSeedValue']

In [25]:
# Lets see the shape and take a sneak peak at the data for all compact results
print(all_compact_results.shape)
all_compact_results.head()


(333293, 11)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,ST,ScoreDiff,HomeAdvantage
0,1985,20,1228,81,1328,64,N,0,S,17,0
1,1985,25,1106,77,1354,70,H,0,S,7,1
2,1985,25,1112,63,1223,56,H,0,S,7,1
3,1985,25,1165,70,1432,54,H,0,S,16,1
4,1985,25,1192,86,1447,74,H,0,S,12,1


In [26]:
# Lets see the shape and take a sneak peak at the data for all detailed results
print(all_detailed_results.shape)
all_detailed_results.head()


(202033, 49)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WFTPct,LFGPct,LFG3Pct,LFTPct,ReboundDiff,AssistDiff,TurnoverDiff,StealDiff,BlockDiff,FoulDiff
0,2003,10,1104,68,1328,62,N,0,27,58,...,0.611111,0.415094,0.200000,0.727273,6,5,5,-2,-1,2
1,2003,10,1272,70,1393,63,N,0,26,62,...,0.526316,0.358209,0.250000,0.450000,-2,9,1,-4,-2,2
2,2003,11,1266,73,1437,61,N,0,24,58,...,0.586207,0.301370,0.115385,0.608696,-10,6,-2,3,-3,2
3,2003,11,1296,56,1457,50,N,0,18,38,...,0.548387,0.367347,0.272727,0.533333,-12,2,-7,10,-1,-5
4,2003,11,1400,77,1208,71,N,0,30,61,...,0.846154,0.387097,0.375000,0.629630,3,0,4,-3,3,6


In [23]:
# Lets see the shape and take a sneak peak at the data for the compact tournament results
print(tourney_compact.shape)
tourney_compact.head()


(4168, 14)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,ST,ScoreDiff,HomeAdvantage,WSeedValue,LSeedValue,SeedDiff
0,1985,136,1116,63,1234,54,N,0,T,9,0,9,8,-1
1,1985,136,1120,59,1345,58,N,0,T,1,0,11,6,-5
2,1985,136,1207,68,1250,43,N,0,T,25,0,1,16,15
3,1985,136,1229,58,1425,55,N,0,T,3,0,9,8,-1
4,1985,136,1242,49,1325,38,N,0,T,11,0,3,14,11


In [24]:
# Lets see the shape and take a sneak peak at the data for the detailed tournament results
print(tourney_detailed.shape)
tourney_detailed.head()

(2276, 52)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTPct,ReboundDiff,AssistDiff,TurnoverDiff,StealDiff,BlockDiff,FoulDiff,WSeedValue,LSeedValue,SeedDiff
0,2003,134,1421,92,1411,84,N,1,32,69,...,0.451613,-1,1,-3,0,3,0,16,16,0
1,2003,136,1112,80,1436,51,N,0,31,66,...,1.000000,13,10,-1,0,4,-7,1,16,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,0.666667,-5,7,-3,5,-1,1,10,7,-3
3,2003,136,1141,79,1166,73,N,0,29,53,...,0.705882,0,-5,-3,7,-5,-2,11,6,-5
4,2003,136,1143,76,1301,74,N,1,27,64,...,0.750000,2,1,-1,3,-6,-5,8,9,1
